In [1]:
#pseudo label -> randomrain
#crash auto label and confidence기반 수작업 -> night
# pip uninstall opencv-python-headless==4.5.5.62
#pip install opencv-python-headless==4.5.2.52
#torch.cuda.empty_cache()

In [2]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from decord import VideoReader

from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from einops import rearrange

from typing import Dict
from pytorchvideo.transforms.transforms_factory import create_video_transform

/home/server-003/anaconda3/envs/competition/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:5: UserWarning: The _functional_video module is deprecated. Please use the functional module instead.
  warnings.warn(
/home/server-003/anaconda3/envs/competition/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The _transforms_video module is deprecated. Please use the transforms module instead.
  warnings.warn(


In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import KFold,StratifiedKFold
import albumentations as A

from datetime import datetime
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from segmentation_models_pytorch.losses import FocalLoss
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import random
import os
import cv2
from tqdm import tqdm
from transformers import AutoModel, AutoImageProcessor, AutoConfig

import warnings

warnings.filterwarnings("ignore")


In [4]:
### https://github.com/davda54/sam

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [6]:
import pandas as pd
all_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [7]:
all_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [8]:
all_df['label'].value_counts()

0     1783
1      318
7      317
3       78
2       51
9       34
11      33
8       30
5       28
4       13
12       6
10       4
6        3
Name: label, dtype: int64

In [9]:

CFG = {
    'VIDEO_LENGTH':10, 
    'IMG_SIZE':235,
    'EPOCHS':4,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':2,
    'SEED':2023,
    'SPLIT':5,
    'ROOT':'./data',
    'MODEL':'MCG-NJU/videomae-base-finetuned-ssv2',
    'NUM_ASB':4,
}

In [10]:
skf = StratifiedKFold(n_splits = CFG['SPLIT'])

In [11]:
all_df['video_path'] = all_df['video_path'].apply(lambda x:CFG['ROOT']+x[1:])
test_df['video_path'] = test_df['video_path'].apply(lambda x:CFG['ROOT']+x[1:])



In [12]:
all_df

,sample_id,video_path,label
0,TRAIN_0000,./data/train/TRAIN_0000.mp4,7
1,TRAIN_0001,./data/train/TRAIN_0001.mp4,7
2,TRAIN_0002,./data/train/TRAIN_0002.mp4,0
3,TRAIN_0003,./data/train/TRAIN_0003.mp4,0
4,TRAIN_0004,./data/train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./data/train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./data/train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./data/train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./data/train/TRAIN_2696.mp4,0


In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [14]:
from transformers import VideoMAEConfig, VideoMAEModel
from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from transformers import XCLIPVisionModel, XCLIPVisionConfig


configuration = VideoMAEConfig()
configuration = AutoConfig.from_pretrained(CFG['MODEL'])
configuration.num_frames = CFG['VIDEO_LENGTH']
configuration.image_size=CFG['IMG_SIZE']
configuration.id2label = {0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',10:'10',11:'11',12:'12'}
configuration.label2id = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'11':11,'12':12}
vae_model = VideoMAEForVideoClassification.from_pretrained(CFG['MODEL'],config=configuration,ignore_mismatched_sizes=True)
# configuration.id2label = {
#         -1:[-1,-1,-1,-1],
#         0:[0,0,0,0],
#         1:[1,1,1,1],
#         2:[1,1,1,2],
#         3:[1,1,2,1],
#         4:[1,1,2,2],
#         5:[1,1,3,1],
#         6:[1,1,3,2],
#         7:[1,2,1,1],
#         8:[1,2,1,2],
#         9:[1,2,2,1],
#         10:[1,2,2,2],
#         11:[1,2,3,1],
#         12:[1,2,3,2]
#     }
# configuration.label2id:{
#         (0,0,0,0):0,
#         (1,1,1,1):1,
#         (1,1,1,2):2,
#         (1,1,2,1):3,
#         (1,1,2,2):4,
#         (1,1,3,1):5,
#         (1,1,3,2):6,
#         (1,2,1,1):7,
#         (1,2,1,2):8,
#         (1,2,2,1):9,
#         (1,2,2,2):10,
#         (1,2,3,1):11,
#         (1,2,3,2):12,
#     }

image_processor_config = AutoImageProcessor.from_pretrained(CFG['MODEL'])



Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-ssv2 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([174, 768]) in the checkpoint and torch.Size([13, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([174]) in the checkpoint and torch.Size([13]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [15]:
# model_name = "slowfast_r50"
# model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

In [16]:
vae_model.config

VideoMAEConfig {
  "_name_or_path": "MCG-NJU/videomae-base-finetuned-ssv2",
  "architectures": [
    "VideoMAEForVideoClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7",
    "8": "8",
    "9": "9",
    "10": "10",
    "11": "11",
    "12": "12"
  },
  "image_size": 235,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1,
    "10": 10,
    "11": 11,
    "12": 12,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9
  },
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_fra

In [17]:
Alb = A.Compose([
        A.Resize(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.Blur(blur_limit=3,p=0.3),
            A.GaussNoise(p=0.3,var_limit=(0, 26)),
            A.Downscale(p=0.3,scale_min=0.7, scale_max=0.99, interpolation=2),
#             A.RandomBrightness(p=0.2, limit=0.05),    
            A.CoarseDropout(p=0.2, max_holes=10, max_height=8, max_width=8, min_holes=5, min_height=2, min_width=2),
        ], p=0.7),
#         A.OneOf([
#         A.ElasticTransform(p=0.3),
#         A.SafeRotate(limit=45,p=0.3),
#         ],p=0.7),
        
        A.Normalize(mean=tuple(image_processor_config.image_mean)
                   ,std=tuple(image_processor_config.image_std),p=1)
    ], p=1)


def aug_video(vid, tfms):
    aug_vid = []
    for x in vid:
        aug_vid.append((tfms(image = np.asarray(x)))['image'])
    return torch.from_numpy(np.stack(aug_vid))

In [18]:
train_transform = create_video_transform(
    mode='train',
    num_samples=CFG['VIDEO_LENGTH'],
    video_mean = tuple(image_processor_config.image_mean),
    video_std = tuple(image_processor_config.image_std),
    crop_size = tuple(image_processor_config.crop_size.values())
)

val_transform = create_video_transform(
    mode='val',
    num_samples=CFG['VIDEO_LENGTH'],
    video_mean = tuple(image_processor_config.image_mean),
    video_std = tuple(image_processor_config.image_std),
    crop_size = tuple(image_processor_config.crop_size.values())
)

In [19]:
class VideoDataset(Dataset):
    def __init__(self, video_path_list, label_list,transform=None):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.Alb = transform
    
    def get_labels(self):   
        return self.label_list  
    
    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])
        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames
        
    def __len__(self):
        return len(self.video_path_list)
    
    def get_video(self, path):
        frames = []
        vr = VideoReader(path)
        video = torch.from_numpy(vr.get_batch(range(50)).asnumpy())
        video = rearrange(video, 't h w c -> c t h w')
#         if self.transform:
#             video = self.transform(video)
        video = rearrange(video, 'c t h w -> t c h w')
        return video
#         cap = cv2.VideoCapture(path)
        
#         for idx in range(50):
#             if idx%5 == 3:
#                 _, img = cap.read()
#                 img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
#                 frames.append(img)
#         if self.Alb is not None:
#             frames = aug_video(frames, tfms=self.Alb)
#         return torch.FloatTensor(np.array(frames)).permute(0, 3, 1, 2)


In [20]:
def train(skf_idx, model, optimizer, train_loader, val_loader, scheduler, device, cls_type):
    model.to(device)
    criterion = FocalLoss('multiclass')
    best_val_score = 0
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
      
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(videos)
#             print(output.armax(-1).item())
            
#             output = F.softmax(output,dim=1)[:1]
#             print(output)
#             print("==")
#             print(labels)
            loss = criterion(output, labels)
#             loss =FocalLoss(gamma=0)(output,labels)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
       
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
      
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            date=datetime.today().strftime("%m_%d_%H_%M")
            torch.save(best_model.state_dict(), './'+cls_type + '_' + str(skf_idx) +'_'+ date + '_best_model.pth')
        skf_idx+=1
    return best_model,achieve,skf_idx

In [21]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)
            
            output = model(videos)
            
            loss = criterion(output.logits, labels)
            
            val_loss.append(loss.item())
            
            preds += output.logits.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [22]:

# checkpoint = torch.load('./checkpoint/crash/crash_4_02_21_16_03_best_model.pth')
# crash_model.load_state_dict(checkpoint)


cls_type = 'all'
base_optimizer = torch.optim.SGD  # define an optimizer for the "sharpness-aware" update
optimizer = SAM(vae_model.parameters(), base_optimizer, lr=CFG["LEARNING_RATE"], momentum=0.5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3,threshold_mode='abs',min_lr=1e-12, verbose=True)
skf_idx=1
for train_idx,val_idx in skf.split(all_df['video_path'],all_df['label']):
    train_dataset = VideoDataset(all_df['video_path'][train_idx].values, all_df['label'][train_idx].values,transform=train_transform)
    val_dataset = VideoDataset(all_df['video_path'][val_idx].values, all_df['label'][val_idx].values, transform=val_transform)
    train_loader = DataLoader(train_dataset,sampler=ImbalancedDatasetSampler(train_dataset),shuffle=False,batch_size = CFG['BATCH_SIZE'],  num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], num_workers=4)
    vae_model,achieve,skf_idx= train(skf_idx,vae_model, optimizer, train_loader, val_loader, scheduler, device, cls_type)


  0%|                                                                                                     | 0/1079 [00:00<?, ?it/s]


ValueError: Input image size (720*1280) doesn't match model (235*235).

In [ ]:
test_dataset = VideoDataset(test_df['video_path'].values,label_list= None, transform=Alb)
test_loader = DataLoader(test_dataset, shuffle=False,batch_size = CFG['BATCH_SIZE']*2,  num_workers=4)
   

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            output = model(videos)
#             preds += output.logits.argmax(1).detach().cpu().numpy().tolist()
            preds += output.logits.detach().cpu().numpy().tolist()

    return preds

In [ ]:
def generate_label(pred,ref_df):
    generate_data = pd.DataFrame(pred,columns = ['one','two'])
    generate_data_index = generate_data[abs(generate_data['one']-generate_data['two'])>CFG['DATA_GENERATE_THRES']].index.values.tolist()
    generate_data_list = generate_data[abs(generate_data['one']-generate_data['two'])>CFG['DATA_GENERATE_THRES']].values.tolist()
    label = np.array(generate_data_list).argmax(1)
    path=ref_df['video_path'][generate_data_index].values.tolist()
    g_df = pd.DataFrame(columns = ['video_path','label'])
    g_df['video_path'] = path
    g_df['label'] = label
    return g_df
# df.max(axis = 1, numeric_only = True)
def generate_weather_label(pred,ref_df):
    generate_data = pd.DataFrame(pred,columns = ['one','two','three'])
    generate_data_index = generate_data[generate_data.max(axis=1,numeric_only=True)>2].index.values.tolist()
    generate_data_list = generate_data[generate_data.max(axis=1,numeric_only=True)>2].values.tolist()

    label = np.array(generate_data_list).argmax(1)
    path=ref_df['video_path'][generate_data_index].values.tolist()
    g_df = pd.DataFrame(columns = ['video_path','label'])
    g_df['video_path'] = path
    g_df['label'] = label
    return g_df



In [ ]:
preds_list = []

preds=None


for idx in range(CFG['NUM_ASB']):
    preds = inference(vae_model, test_loader, device)
    preds_list.append(preds)
    
pred_sum = np.sum(crash_preds_list,axis=0)
pred_max = crash_pred_sum.argmax(1).tolist()







In [ ]:
# # all_df['label'].values.tolist()
# gt = crash_df['label'].values.tolist()
# f1_score(gt,crash_max,average='macro')
pred_max

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
# comp = pd.read_csv('./submit/2023_02_21_10_44_04.csv')

In [ ]:
submit['label'] = preds

In [ ]:
# comp['label'].value_counts()

In [ ]:
submit['label'].value_counts()

In [ ]:
date=datetime.today().strftime("%Y_%m_%d_%H_%M_%S")
submit.to_csv('./'+date+'.csv', index=False)